<a href="https://colab.research.google.com/github/BrendaLoznik/waterpumps/blob/main/Waterpumps.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

competition: https://www.drivendata.org/competitions/7/pump-it-up-data-mining-the-water-table/

geographical data: https://heartbeat.fritz.ai/working-with-geospatial-data-in-machine-learning-ad4097c7228d

###**1.1 Loading Libraries**

In [1]:
#basic libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from scipy import stats

###**1.2 Loading data**

In [3]:
from google.colab import drive 
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [4]:
train_labels = pd.read_csv('/content/gdrive/MyDrive/Kaggle/Waterpumps/train_labels.csv')
train_values = pd.read_csv('/content/gdrive/MyDrive/Kaggle/Waterpumps/train_values.csv', parse_dates = ['date_recorded'])
test_values = pd.read_csv('/content/gdrive/MyDrive/Kaggle/Waterpumps/test_values.csv', parse_dates = ['date_recorded'])

In [11]:
#merge train values and train labels to single dataset
train= pd.merge(train_labels , train_values, on='id')


,id,status_group,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,69572,functional,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,Lake Nyasa,Mnyusi B,Iringa,11,5,Ludewa,Mundindi,109,True,GeoData Consultants Ltd,VWC,Roman,False,1999,gravity,gravity,gravity,vwc,user-group,pay annually,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
1,8776,functional,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,Lake Victoria,Nyamara,Mara,20,2,Serengeti,Natta,280,NaN,GeoData Consultants Ltd,Other,NaN,True,2010,gravity,gravity,gravity,wug,user-group,never pay,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
2,34310,functional,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,Pangani,Majengo,Manyara,21,4,Simanjiro,Ngorika,250,True,GeoData Consultants Ltd,VWC,Nyumba ya mungu pipe scheme,True,2009,gravity,gravity,gravity,vwc,user-group,pay per bucket,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe
3,67743,non functional,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,Ruvuma / Southern Coast,Mahakamani,Mtwara,90,63,Nanyumbu,Nanyumbu,58,True,GeoData Consultants Ltd,VWC,NaN,True,1986,submersible,submersible,submersible,vwc,user-group,never pay,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe
4,19728,functional,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,Lake Victoria,Kyanyamisa,Kagera,18,1,Karagwe,Nyakasimbi,0,True,GeoData Consultants Ltd,NaN,NaN,True,0,gravity,gravity,gravity,other,other,never pay,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe


In [16]:
#concatenate train and test for easier data minipulation
raw = pd.concat([train, test_values])
raw.head()

,id,status_group,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,69572,functional,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,Lake Nyasa,Mnyusi B,Iringa,11,5,Ludewa,Mundindi,109,True,GeoData Consultants Ltd,VWC,Roman,False,1999,gravity,gravity,gravity,vwc,user-group,pay annually,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
1,8776,functional,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,Lake Victoria,Nyamara,Mara,20,2,Serengeti,Natta,280,NaN,GeoData Consultants Ltd,Other,NaN,True,2010,gravity,gravity,gravity,wug,user-group,never pay,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
2,34310,functional,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,Pangani,Majengo,Manyara,21,4,Simanjiro,Ngorika,250,True,GeoData Consultants Ltd,VWC,Nyumba ya mungu pipe scheme,True,2009,gravity,gravity,gravity,vwc,user-group,pay per bucket,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe
3,67743,non functional,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,Ruvuma / Southern Coast,Mahakamani,Mtwara,90,63,Nanyumbu,Nanyumbu,58,True,GeoData Consultants Ltd,VWC,NaN,True,1986,submersible,submersible,submersible,vwc,user-group,never pay,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe
4,19728,functional,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,Lake Victoria,Kyanyamisa,Kagera,18,1,Karagwe,Nyakasimbi,0,True,GeoData Consultants Ltd,NaN,NaN,True,0,gravity,gravity,gravity,other,other,never pay,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe


In [95]:
#convert faulty datatypes
raw[['region_code','district_code', 'num_private']] = raw[['region_code','district_code', 'num_private']].astype('object')

In [96]:
#divide features into numerical and cateogorical for ease of describtion
numerical = ['id', 'amount_tsh', 'date_recorded', 'gps_height',  'longitude', 'latitude', 
             'population', 'construction_year' ]

categorical = ['status_group', 'funder', 'installer','wpt_name','num_private',  'basin', 'subvillage', 'region', 'region_code',
               'district_code', 'lga', 'ward', 'public_meeting', 'recorded_by', 'scheme_management', 
               'scheme_name', 'permit', 'extraction_type', 'extraction_type_group','extraction_type_class',
               'management', 'management_group', 'payment','payment_type', 'water_quality', 'quality_group', 'quantity',
               'quantity_group', 'source', 'source_type', 'source_class', 'waterpoint_type', 'waterpoint_type_group']


print("Number of numerical features: {}".format(len(numerical)))
print("Number of categorical features: {}".format(len(categorical)))


Number of numerical features: 8
Number of categorical features: 33


###**2.1 Exploring the data**

####**2.1.1 Numerical data**

In [97]:
raw[numerical].head()

,id,amount_tsh,date_recorded,gps_height,longitude,latitude,population,construction_year
0,69572,6000.0,2011-03-14,1390,34.938093,-9.856322,109,1999
1,8776,0.0,2013-03-06,1399,34.698766,-2.147466,280,2010
2,34310,25.0,2013-02-25,686,37.460664,-3.821329,250,2009
3,67743,0.0,2013-01-28,263,38.486161,-11.155298,58,1986
4,19728,0.0,2011-07-13,0,31.130847,-1.825359,0,0


In [98]:
raw[numerical].describe()

,id,amount_tsh,gps_height,longitude,latitude,population,construction_year
count,74250.000000,74250.000000,74250.000000,74250.000000,7.425000e+04,74250.000000,74250.000000
mean,37124.500000,318.685704,665.667313,34.074262,-5.701771e+00,180.750828,1298.463650
std,21434.273081,2906.762364,692.761033,6.572519,2.944969e+00,471.086120,952.349375
min,0.000000,0.000000,-90.000000,0.000000,-1.164944e+01,0.000000,0.000000
25%,18562.250000,0.000000,0.000000,33.086819,-8.525675e+00,0.000000,0.000000
50%,37124.500000,0.000000,364.000000,34.907475,-5.026540e+00,25.000000,1986.000000
75%,55686.750000,20.000000,1317.000000,37.181685,-3.325058e+00,215.000000,2004.000000
max,74249.000000,350000.000000,2777.000000,40.345193,-2.000000e-08,30500.000000,2013.000000


In [99]:
#print percentage of missing numerical datapoints
round(raw[numerical].isna().sum()*100/raw[numerical].isna().count()).sort_values(ascending=False)

construction_year    0.0
population           0.0
latitude             0.0
longitude            0.0
gps_height           0.0
date_recorded        0.0
amount_tsh           0.0
id                   0.0
dtype: float64

In [100]:
#explore the cardinality of numerical features
for column in raw[numerical]:
  unique = raw[column].nunique()
  print(column,":", unique)
 

id : 74250
amount_tsh : 102
date_recorded : 369
gps_height : 2456
longitude : 71870
latitude : 71869
population : 1128
construction_year : 55


####**2.1.1 Categorical data**

In [101]:
raw[categorical].head()

,status_group,funder,installer,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,public_meeting,recorded_by,scheme_management,scheme_name,permit,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,functional,Roman,Roman,none,0,Lake Nyasa,Mnyusi B,Iringa,11,5,Ludewa,Mundindi,True,GeoData Consultants Ltd,VWC,Roman,False,gravity,gravity,gravity,vwc,user-group,pay annually,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
1,functional,Grumeti,GRUMETI,Zahanati,0,Lake Victoria,Nyamara,Mara,20,2,Serengeti,Natta,NaN,GeoData Consultants Ltd,Other,NaN,True,gravity,gravity,gravity,wug,user-group,never pay,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
2,functional,Lottery Club,World vision,Kwa Mahundi,0,Pangani,Majengo,Manyara,21,4,Simanjiro,Ngorika,True,GeoData Consultants Ltd,VWC,Nyumba ya mungu pipe scheme,True,gravity,gravity,gravity,vwc,user-group,pay per bucket,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe
3,non functional,Unicef,UNICEF,Zahanati Ya Nanyumbu,0,Ruvuma / Southern Coast,Mahakamani,Mtwara,90,63,Nanyumbu,Nanyumbu,True,GeoData Consultants Ltd,VWC,NaN,True,submersible,submersible,submersible,vwc,user-group,never pay,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe
4,functional,Action In A,Artisan,Shuleni,0,Lake Victoria,Kyanyamisa,Kagera,18,1,Karagwe,Nyakasimbi,True,GeoData Consultants Ltd,NaN,NaN,True,gravity,gravity,gravity,other,other,never pay,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe


In [102]:
raw[categorical].describe(include='all')

,status_group,funder,installer,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,public_meeting,recorded_by,scheme_management,scheme_name,permit,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
count,59400,69746,69718,74250,74250,74250,73780,74250,74250,74250,74250,74250,70095,74250,69404,38992,70457,74250,74250,74250,74250,74250,74250,74250,74250,74250,74250,74250,74250,74250,74250,74250,74250
unique,3,2140,2410,45684,68,9,21425,21,27,20,125,2098,2,1,12,2868,2,18,13,7,12,5,7,7,8,6,5,5,10,7,3,7,6
top,functional,Government Of Tanzania,DWE,none,0,Lake Victoria,Shuleni,Iringa,11,1,Njombe,Igosi,True,GeoData Consultants Ltd,VWC,K,True,gravity,gravity,gravity,vwc,user-group,never pay,never pay,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
freq,32259,11299,21751,4440,73299,12871,646,6599,6608,15299,3128,386,63749,74250,45917,858,48606,33263,33263,33263,50624,65538,31712,31712,63505,63505,41522,41522,21216,21216,57286,35628,43239


In [91]:
# print absolute number  of missing continious data points
#print(raw[categorical].isna().sum().sort_values(ascending=False))

In [88]:
#print percentage of missing numerical datapoints
round(raw[categorical].isna().sum()*100/raw[categorical].isna().count()).sort_values(ascending=False)

scheme_name              47.0
status_group             20.0
scheme_management         7.0
funder                    6.0
installer                 6.0
public_meeting            6.0
permit                    5.0
subvillage                1.0
quality_group             0.0
water_quality             0.0
source_class              0.0
source_type               0.0
wpt_name                  0.0
num_private               0.0
basin                     0.0
region                    0.0
region_code               0.0
district_code             0.0
lga                       0.0
ward                      0.0
source                    0.0
recorded_by               0.0
quantity_group            0.0
quantity                  0.0
waterpoint_type           0.0
extraction_type           0.0
extraction_type_group     0.0
extraction_type_class     0.0
management                0.0
management_group          0.0
payment                   0.0
payment_type              0.0
waterpoint_type_group     0.0
dtype: flo

In [87]:
#value = raw['status_group'].value_counts()
#value

In [76]:
#explore the cardinality of numerical features
for column in raw[categorical]:
  mode = raw[column].value_counts()
  print(mode)
 

functional                 32259
non functional             22824
functional needs repair     4317
Name: status_group, dtype: int64
Government Of Tanzania          11299
Danida                           3907
Hesawa                           2782
Rwssp                            1703
World Bank                       1701
                                ...  
Idf                                 1
Mahita                              1
Wdsp                                1
Nyeisa                              1
Mwalimu  Maneromango Muhenzi        1
Name: funder, Length: 2140, dtype: int64
DWE                            21751
Government                      2282
RWE                             1498
Commu                           1347
DANIDA                          1305
                               ...  
Saidi Halfani                      1
OMARY MONA                         1
KARUMBA BIULDING CONTRACTOR        1
KKKT Katiti juu                    1
J mal Abdallah                     1
Na

In [106]:
#how many unique gps coordinates are there?
#temp = raw.copy()
#temp['location'] = temp['longitude'].astype(str) + '-' + temp['latitude'].astype(str)
#temp['location'].nunique()


,id,status_group,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,location
0,69572,functional,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,Lake Nyasa,Mnyusi B,Iringa,11,5,Ludewa,Mundindi,109,True,GeoData Consultants Ltd,VWC,Roman,False,1999,gravity,gravity,gravity,vwc,user-group,pay annually,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,34.93809275--9.85632177
1,8776,functional,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,Lake Victoria,Nyamara,Mara,20,2,Serengeti,Natta,280,NaN,GeoData Consultants Ltd,Other,NaN,True,2010,gravity,gravity,gravity,wug,user-group,never pay,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,34.6987661--2.14746569
2,34310,functional,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,Pangani,Majengo,Manyara,21,4,Simanjiro,Ngorika,250,True,GeoData Consultants Ltd,VWC,Nyumba ya mungu pipe scheme,True,2009,gravity,gravity,gravity,vwc,user-group,pay per bucket,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe,37.46066446--3.82132853
3,67743,non functional,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,Ruvuma / Southern Coast,Mahakamani,Mtwara,90,63,Nanyumbu,Nanyumbu,58,True,GeoData Consultants Ltd,VWC,NaN,True,1986,submersible,submersible,submersible,vwc,user-group,never pay,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe,38.48616088--11.15529772
4,19728,functional,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,Lake Victoria,Kyanyamisa,Kagera,18,1,Karagwe,Nyakasimbi,0,True,GeoData Consultants Ltd,NaN,NaN,True,0,gravity,gravity,gravity,other,other,never pay,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,31.13084671--1.82535885
